In [1]:
import os
import argparse
import datetime

import torch
from torch import nn
import numpy as np
import gymnasium as gym

from tianshou.data import Collector, Batch, to_torch
from tianshou.data.types import RolloutBatchProtocol
from tianshou.data.buffer.vecbuf import VectorReplayBuffer, ReplayBuffer
from tianshou.env import SubprocVectorEnv
from tianshou.policy import SACPolicy, BasePolicy
from tianshou.utils.net.common import Net
from tianshou.utils.net.continuous import ActorProb, Critic
from examples.offline.utils import load_buffer_d4rl
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger
from tianshou.trainer import OffpolicyTrainer

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:596: UserWarning: WARN: plugin: shimmy.registration:register_gymnasium_envs raised Traceback (most recent call last):
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py", line 594, in load_plugin_envs
    fn()
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shimmy/registration.py", line 262, in register_gymnasium_envs
    _register_dm_control_envs()
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shimmy/registration.py", line 26, in _register_dm_control_envs
    from shimmy.dm_control_compatibility import DmControlCompatibilityV0
  File "/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/shim

In [2]:
device = "cuda:3"

In [3]:
def get_sac_args():
    args = argparse.Namespace(
        task="Hopper-v2",
        buffer_size=1000000,
        hidden_sizes=[256, 256, 256],
        actor_lr=1e-4,
        critic_lr=3e-4,
        gamma=0.99,
        tau=0.005,
        alpha_lr=1e-4,
        alpha=0.36,
        start_timesteps=1,
        epoch=200,
        step_per_epoch=5000,
        step_per_collect=1,
        update_per_step=1,
        batch_size=256,
        training_num=1,
        test_num=10,
        device=device,
        norm_layer=True,
    )
    return args

In [4]:
args = get_sac_args()
env = gym.make(args.task)

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(


In [5]:
def load_policy(path):
    args = get_sac_args()
    env = gym.make(args.task)
    args.state_shape = env.observation_space.shape or env.observation_space.n
    args.action_shape = env.action_space.shape or env.action_space.n
    args.max_action = env.action_space.high[0]
    # model
    net_a = Net(args.state_shape, hidden_sizes=args.hidden_sizes, device=args.device)
    actor = ActorProb(
        net_a,
        args.action_shape,
        device=args.device,
        unbounded=True,
        conditioned_sigma=True,
    ).to(args.device)
    actor_optim = torch.optim.Adam(actor.parameters(), lr=args.actor_lr)
    net_c1 = Net(
        args.state_shape,
        args.action_shape,
        hidden_sizes=args.hidden_sizes,
        concat=True,
        device=args.device,
        norm_layer=nn.LayerNorm if args.norm_layer else None
    )
    net_c2 = Net(
        args.state_shape,
        args.action_shape,
        hidden_sizes=args.hidden_sizes,
        concat=True,
        device=args.device,
        norm_layer=nn.LayerNorm if args.norm_layer else None
    )
    critic1 = Critic(net_c1, device=args.device).to(args.device)
    critic1_optim = torch.optim.Adam(critic1.parameters(), lr=args.critic_lr)
    critic2 = Critic(net_c2, device=args.device).to(args.device)
    critic2_optim = torch.optim.Adam(critic2.parameters(), lr=args.critic_lr)
    target_entropy = -np.prod(env.action_space.shape)
    log_alpha = torch.tensor([np.log(args.alpha)], requires_grad=True, device=device)
    alpha_optim = torch.optim.Adam([log_alpha], lr=args.alpha_lr)
    alpha = (target_entropy, log_alpha, alpha_optim)
    policy = SACPolicy(
        actor,
        actor_optim,
        critic1,
        critic1_optim,
        critic2,
        critic2_optim,
        tau=args.tau,
        gamma=args.gamma,
        alpha=alpha,
        action_space=env.action_space,
    )
    policy.load_state_dict(torch.load(path, map_location=args.device))
    print("Loaded agent from: ", path)
    return policy

In [19]:
cql_policy = load_policy("/data/user/R901105/dev/log/Hopper-v2/cql/0/231220-111219/policy.pth")

Loaded agent from:  /data/user/R901105/dev/log/Hopper-v2/cql/0/231220-111219/policy.pth


/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(


In [7]:
sac_policy = load_policy("/data/user/R901105/dev/log/Hopper-v2/sac/0/231219-163624/policy.pth")

Loaded agent from:  /data/user/R901105/dev/log/Hopper-v2/sac/0/231219-163624/policy.pth


/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(


In [8]:
envs = SubprocVectorEnv([lambda: gym.make(args.task) for _ in range(10)])

In [9]:
cql_policy.eval()
cql_collector = Collector(cql_policy, envs)
cql_result = cql_collector.collect(n_episode=40)
cql_result

{'n/ep': 40,
 'n/st': 26003,
 'rews': array([1486.25269722, 1823.26179735, 1815.41471127, 1823.64823606,
        2087.71584701, 2396.16515738, 2461.0877074 , 2686.45698862,
        2770.80036959, 3384.23060393, 1842.48922955, 1866.94245461,
        1898.13672459, 1807.75406455, 1417.38894988, 1665.01841421,
        2181.79117767, 1902.67316167, 3142.5463871 , 1484.83423753,
        2180.77193707, 2174.34573781, 1804.64465422, 2182.29790974,
        1899.60368569, 2745.06289326, 2078.45634837, 2320.67582828,
        1856.80881647, 2773.04795628, 2183.64806295, 2268.10985287,
        1907.08551018, 1816.56290292, 1843.81815116, 2998.18710922,
        1994.10289773, 2978.34000772, 2359.09940191, 2161.83989175]),
 'lens': array([455, 546, 551, 551, 633, 717, 745, 798, 829, 998, 551, 562, 575,
        546, 436, 505, 655, 575, 938, 454, 657, 655, 548, 656, 575, 822,
        619, 696, 556, 833, 659, 683, 574, 551, 557, 891, 600, 890, 711,
        650]),
 'idxs': array([1, 7, 4, 6, 2, 3, 0, 8,

In [10]:
sac_policy.eval()
sac_collector = Collector(sac_policy, envs)
sac_result = sac_collector.collect(n_episode=40)
sac_result

{'n/ep': 40,
 'n/st': 38521,
 'rews': array([1650.7395058 , 2593.72564536, 3602.29768638, 3553.53303002,
        3580.08063385, 3579.30580244, 3520.1980104 , 3553.8322047 ,
        3550.06542435, 3569.61099467, 3607.30669895, 3578.27491201,
        3528.71862059, 3586.13532283, 3559.06398147, 3580.10912595,
        3571.73881482, 3570.89898834, 3521.56364915, 3546.21762465,
        3533.51493836, 2312.71887543, 3524.73381178, 3234.52668798,
        3129.11860956, 3535.58325009, 3591.21120569, 3570.74677282,
        3568.84516274, 3556.83502317, 3581.03732314, 3564.99177602,
        3575.07730797, 3533.45569568, 3550.81724913, 3535.46933535,
        3562.03007878, 3568.32463834, 3497.97434734, 3642.65175359]),
 'lens': array([ 478,  702, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,  641,
         962,  861,  877, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000]),

In [81]:
class MixedPolicy(BasePolicy):
    def __init__(self, base_policy: BasePolicy, expert_policy: BasePolicy, action_space, buffer):
        super().__init__(action_space=action_space, action_scaling=True)
        self.base_policy = base_policy
        self.expert_policy = expert_policy
        self.device = device
        self.buffer = buffer

    def forward(self, batch: RolloutBatchProtocol, state=None, **kwargs):
        bsz = len(batch.obs)
        batch = to_torch(batch, dtype=torch.float32, device=self.device)
        self.base_policy.eval()
        self.expert_policy.eval()
        with torch.no_grad():
            expert_result = self.expert_policy(batch)
            base_result = self.base_policy(batch)
            expert_qvalues1 = self.expert_policy.critic1(batch.obs, expert_result.act)
            expert_qvalues2 = self.expert_policy.critic2(batch.obs, expert_result.act)
            expert_qvalues = torch.minimum(expert_qvalues1, expert_qvalues2)
            base_qvalues1 = self.base_policy.critic1(batch.obs, base_result.act)
            base_qvalues2 = self.base_policy.critic2(batch.obs, base_result.act)
            base_qvalues = torch.minimum(base_qvalues1, base_qvalues2)
        cede_ctrl = base_qvalues < expert_qvalues
        actions = torch.where(cede_ctrl, expert_result.act, base_result.act)
        return Batch(**{'act': actions, 'policy': Batch({'cede_ctrl': cede_ctrl})})

    def train(self, mode: bool = True) -> "MixedPolicy":
        self.base_policy.train(mode)
        return self
    
    def process_fn(self, batch: RolloutBatchProtocol, buffer: ReplayBuffer, indices: np.ndarray) -> RolloutBatchProtocol:
        return self.base_policy.process_fn(batch, buffer, indices)

    def learn(self, batch, **kwargs):
        # cede_ctrl = batch.policy.cede_ctrl.cpu().squeeze()
        # train_batch = batch[~cede_ctrl]
        # info = self.base_policy.learn(train_batch)
        info = self.base_policy.learn(batch)
        return info

In [11]:
test_buffer = VectorReplayBuffer(5000, 5)

In [82]:
mixed_policy = MixedPolicy(cql_policy, sac_policy, env.action_space, test_buffer)

In [12]:
mixed_collector = Collector(mixed_policy, envs)
mixed_result = mixed_collector.collect(n_episode=40)
mixed_result

{'n/ep': 40,
 'n/st': 39442,
 'rews': array([2877.85420345, 3579.38976417, 3566.2887685 , 3612.26257751,
        3648.73333248, 3655.45149677, 3559.89113674, 3552.6419363 ,
        3588.0665408 , 3578.10143437, 3576.98877615, 3561.46943533,
        3558.70658515, 3598.58161738, 3558.16991895, 3525.51417879,
        3579.98865515, 3558.74737203, 3518.58784384, 3641.33416581,
        3563.76800552, 3231.92112228, 3558.71492095, 3535.73037386,
        3558.96929922, 3583.61728139, 3556.20513785, 3542.30987626,
        3584.5683532 , 3573.77252436, 2903.35971768, 3574.50541953,
        3551.28549583, 3540.74278544, 3513.75433021, 3546.94601085,
        3569.82593565, 3520.37927402, 3546.47417846, 3562.03266267]),
 'lens': array([ 798, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,  869,
        1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000,  775, 1000, 1000,
        1000, 1000, 1000, 1000, 1000, 1000, 1000]),

In [59]:
class DeterministicPolicy(BasePolicy):
    def __init__(self, policy, action_space):
        super().__init__(action_space=action_space, action_scaling=True)
        self.policy = policy

    def train(self, mode: bool = True) -> "DeterministicPolicy":
        self.policy.train(mode)
        return self
    
    def forward(self, batch, state=None, **kwargs):
        self.policy.eval()
        return self.policy(batch)
    
    def learn(self, batch, **kwargs):
        info = self.policy.learn(batch)
        return info
    
    def process_fn(self, batch: RolloutBatchProtocol, buffer: ReplayBuffer, indices: np.ndarray) -> RolloutBatchProtocol:
        return self.policy.process_fn(batch, buffer, indices)

In [60]:
det_cql = DeterministicPolicy(cql_policy, env.action_space)

In [ ]:
det_cql_collector = Collector(cql_policy, envs)
cql_result = cql_collector.collect(n_episode=40)
cql_result

In [13]:
test_envs = SubprocVectorEnv([lambda: gym.make(args.task) for _ in range(5)])

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment Hopper-v2 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gymnasium/e

In [18]:
offline_data = load_buffer_d4rl("hopper-medium-v2")

/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gym/envs/mujoco/mujoco_env.py:190: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/data/user/R901105/.conda/envs/dev/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
load datafile:   5%|▍         | 1/21 [00:00<00:02,  9.40it/s]

load datafile: 100%|██████████| 21/21 [00:01<00:00, 13.28it/s]


In [20]:
policy = cql_policy

In [15]:
test_buffer = VectorReplayBuffer(5000, 5)
train_collector = Collector(policy, env, offline_data)
test_collector = Collector(policy, test_envs, test_buffer)

/data/user/R901105/dev/my_fork/tianshou/tianshou/data/collector.py:70: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


In [21]:
# log
now = datetime.datetime.now().strftime("%y%m%d-%H%M%S")
log_name = os.path.join(args.task, policy.__class__.__name__, now)
log_path = os.path.join("/data/user/R901105/dev/log", log_name)
writer = SummaryWriter(log_path)
logger = TensorboardLogger(writer)
print(log_path)

/data/user/R901105/dev/log/Hopper-v2/SACPolicy/240423-173106


In [15]:
def get_returns(policy, return_cede_ctrl=False):
    policy.eval()
    returns = []
    cede_ctrl = []
    for _ in range(5):
        obs, info = env.reset()
        done = False
        cum_reward = 0
        actions = []
        if return_cede_ctrl:
            cctrl_ep = []
        while not done:
            batch = Batch(obs=torch.from_numpy(np.expand_dims(obs, 0)).to(device), info=info)
            with torch.no_grad():
                result = policy(batch)
            act = result.act.cpu().squeeze().numpy()
            act = policy.map_action(act)
            if return_cede_ctrl:
                cctrl_ep.append(result.policy.cede_ctrl.cpu().squeeze().numpy())
            obs, reward, terminated, truncated, info = env.step(act)
            actions.append(act)
            cum_reward += reward
            done = terminated or truncated
        returns.append(cum_reward)
        if return_cede_ctrl:
            cede_ctrl.append(np.array(cctrl_ep).mean())
    if return_cede_ctrl:
        return np.array(returns),  np.array(cede_ctrl).mean()
    return np.array(returns), _

In [16]:
def test_fn(num_epoch: int, step_idx: int):
    returns, cede_ctrl = get_returns(policy, True)
    print(returns.mean(), cede_ctrl.mean())

In [22]:
result = OffpolicyTrainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    buffer=offline_data,
    # test_fn=test_fn,
    max_epoch=200,
    step_per_epoch=1000,
    step_per_collect=1,
    episode_per_test=1,
    batch_size=256,
    logger=logger,
    update_per_step=1,
    test_in_train=False,
).run()

Epoch #1: 1001it [00:15, 64.63it/s, alpha=0.323, env_step=1000, gradient_step=1000, len=600, loss/actor=-265.235, loss/alpha=-1.450, loss/critic1=10.468, loss/critic2=10.492, n/ep=0, n/st=1, rew=2024.22]                          


Epoch #1: test_reward: 3150.884889 ± 0.000000, best_reward: 3150.884889 ± 0.000000 in #1


Epoch #2: 1001it [00:16, 60.38it/s, alpha=0.290, env_step=2000, gradient_step=2000, len=446, loss/actor=-263.216, loss/alpha=-1.732, loss/critic1=9.273, loss/critic2=9.195, n/ep=0, n/st=1, rew=1515.25]                           


Epoch #2: test_reward: 3051.355611 ± 0.000000, best_reward: 3150.884889 ± 0.000000 in #1


Epoch #3: 1001it [00:16, 60.82it/s, alpha=0.261, env_step=3000, gradient_step=3000, len=385, loss/actor=-261.482, loss/alpha=-1.961, loss/critic1=6.918, loss/critic2=7.410, n/ep=0, n/st=1, rew=1276.27]                           


...

Epoch #52: test_reward: 3255.556129 ± 0.000000, best_reward: 3276.466343 ± 0.000000 in #13


Epoch #53: 1001it [00:15, 63.81it/s, alpha=0.035, env_step=53000, gradient_step=53000, len=379, loss/actor=-303.822, loss/alpha=-0.721, loss/critic1=12.893, loss/critic2=12.607, n/ep=0, n/st=1, rew=1243.48]                          


Epoch #53: test_reward: 2444.425899 ± 0.000000, best_reward: 3276.466343 ± 0.000000 in #13


Epoch #54:  51%|#####1    | 514/1000 [00:08<00:08, 58.48it/s, alpha=0.034, env_step=53513, gradient_step=53513, len=1000, loss/actor=-303.479, loss/alpha=-0.610, loss/critic1=2.223, loss/critic2=2.781, n/ep=0, n/st=1, rew=3216.97] 


KeyboardInterrupt: 